In [1]:
from __future__ import unicode_literals  # 이게 뭘까
import requests
import pymongo
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from scrapy.http import TextResponse
import urllib.request
import urllib
import getpass
import time
import re
from google.cloud import vision
from google.cloud.vision import types
import os
import io
from os import walk
from bs4 import BeautifulSoup

## 라프텔 썸네일 크롤링
- 전체 애니메이션 썸네일 긁어오기

In [ ]:
def crawling_laftel_thumbnail():
    df = pd.DataFrame(columns=['title', 'genre', 'content', 'age'])
    webtoon_info = []
    total_img_count = 0

    domain_url = 'https://laftel.net'
    main_url = 'https://laftel.net/finder'

    chrome_options = Options()
    chrome_options.add_argument("--window-size=1600,1080")

    # if headless == True:
    #     chrome_options.add_argument('headless')

    driver = webdriver.Chrome(options=chrome_options)
    driver.get(main_url)
    
    driver.find_element_by_xpath('//*[@id="root"]/div/div[2]/div[2]/div[1]/div/div/div[1]/div/div[2]/section/div[1]').click()
    
    SCROLL_PAUSE_TIME = 0.5
    time.sleep(SCROLL_PAUSE_TIME)

    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        last_height = new_height
        
    if not os.path.exists('./imgs/{}'.format(title)):
                os.mkdir('./imgs/{}'.format(title))
        
    thumbnail_imgs = driver.find_elements_by_xpath('//*[@id="root"]/div/div[2]/div[2]/div[2]/div[2]/div/div/img')
    
    
    

In [2]:
df = pd.DataFrame(columns=['title', 'genre', 'content', 'age'])
webtoon_info = []
total_img_count = 0

domain_url = 'https://laftel.net'
main_url = 'https://laftel.net/finder'

chrome_options = Options()
chrome_options.add_argument("--window-size=1600,1080")

# if headless == True:
#     chrome_options.add_argument('headless')

driver = webdriver.Chrome(options=chrome_options)
driver.get(main_url)

In [25]:
driver.find_element_by_xpath('//*[@id="root"]/div/div[2]/div[2]/div[1]/div/div/div[1]/div/div[2]/section/div[1]').click()

In [6]:
SCROLL_PAUSE_TIME = 0.5

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    time.sleep(SCROLL_PAUSE_TIME)

    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        break
    last_height = new_height

In [26]:
thumbnail_imgs = driver.find_elements_by_xpath('//*[@id="root"]/div/div[2]/div[2]/div[2]/div[2]/div/div/img')
len(thumbnail_img)

20

In [30]:
thumbnail_img

[<selenium.webdriver.remote.webelement.WebElement (session="ce9ec0d4b8c8854f9012456bc790d4d9", element="d100d6fc-ce70-473a-a382-d9bbd09c1717")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ce9ec0d4b8c8854f9012456bc790d4d9", element="d9fbe076-0439-415c-bb0a-1eddc0b28efa")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ce9ec0d4b8c8854f9012456bc790d4d9", element="ef93dd3c-1f8e-4e4d-8921-a6c7e7031573")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ce9ec0d4b8c8854f9012456bc790d4d9", element="cdb37be3-827f-49d5-9b63-6c39fd493604")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ce9ec0d4b8c8854f9012456bc790d4d9", element="7e0cee98-c015-4c6e-ba5d-6af8f5c1e3c2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ce9ec0d4b8c8854f9012456bc790d4d9", element="35ac7483-b25e-48f8-9662-190f8b7427c4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ce9ec0d4b8c8854f9012456bc790d4d9", element="44fe7dea-fefb-4630-a467-0d